## Libraries

In [1]:
import torch
from torch import tensor

## Broadcasting

### Example broadcast

In [2]:
c = tensor([10, 20, 30])
m = tensor([[1., 2, 3], [4, 5, 6], [7, 8, 9]])

In [3]:
c.shape, m.shape

(torch.Size([3]), torch.Size([3, 3]))

In [4]:
c + m

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

### Simple exercise

Suppose this is a batch of RGB images

In [5]:
image_batch = torch.rand(64, 3, 256, 256)
image_batch.shape

torch.Size([64, 3, 256, 256])

Normalize it with vectors of 3 elements. One for the mean and one for std

In [6]:
mean = torch.mean(image_batch, dim=(0, 2, 3))
mean, mean.shape

(tensor([0.4997, 0.5000, 0.5000]), torch.Size([3]))

In [7]:
std = torch.std(image_batch, dim=(0, 2, 3))
std, std.shape

(tensor([0.2886, 0.2887, 0.2886]), torch.Size([3]))

In [8]:
broadcasted_mean = mean.unsqueeze(0).unsqueeze(2).unsqueeze(3)
broadcasted_mean.shape

torch.Size([1, 3, 1, 1])

In [9]:
broadcasted_mean.storage

<bound method Tensor.storage of tensor([[[[0.4997]],

         [[0.5000]],

         [[0.5000]]]])>

In [10]:
broadcasted_std = std.unsqueeze(1).unsqueeze(1)
broadcasted_std.shape

torch.Size([3, 1, 1])

In [11]:
broadcasted_std.storage

<bound method Tensor.storage of tensor([[[0.2886]],

        [[0.2887]],

        [[0.2886]]])>

In [12]:
normalized_images = (image_batch - broadcasted_mean)/broadcasted_std
normalized_images.shape

torch.Size([64, 3, 256, 256])

## Einstein Summation

Compact representation for combining products and sums in a general way. For example:  
ik,kj->ij

Lefthand side represents the "operators" while the right side would be the result's dimensions. The rules of Einsum are:
1. Repeated indices on the left side are implicitly summed over if they are not on the right side
2. Each index can appear at most twice on the left side
3. Unrepeated indices on the left side must appear on the right side

In [24]:
x = torch.randn(2,3)
y = torch.randn(3,2)
x, y

(tensor([[-0.1022, -0.0583, -0.1178],
         [-1.3592,  0.1550, -1.3527]]),
 tensor([[ 0.0110,  0.3932],
         [ 0.7356,  1.3949],
         [-0.0850, -0.8467]]))

### Transpose

In [31]:
torch.einsum("ij->ji", x)

tensor([[-0.1022, -1.3592],
        [-0.0583,  0.1550],
        [-0.1178, -1.3527]])

### Matrix product

In [25]:
torch.einsum('ik, kj -> ij', x, y)

tensor([[-0.0340, -0.0217],
        [ 0.2141,  0.8271]])

### Gram matrix

In [26]:
torch.einsum('ij, kj -> ik', x, x)

tensor([[0.0277, 0.2892],
        [0.2892, 3.7013]])